In [1]:
#!pip install transformers

In [3]:
import notebook

In [69]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = "distilbert-base-uncased"
# Load a tokenizer (e.g., BERT-based)
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [71]:
from datasets import Dataset, DatasetDict

In [73]:
if notebook.__version__== '6.5.2':
    import ipywidgets as widgets
    from IPython.display import display

    # Create file upload widget
    uploader = widgets.FileUpload(accept='', multiple=False)  # Accept all files, allow single upload

    # Display widget
    display(uploader)

In [75]:
# Does not work with Jupyter Notebook version 7.0.8

if notebook.__version__== '6.5.2':
    import pandas as pd
    from pathlib import Path
    import io

    if uploader.value:
        uploaded_file = next(iter(uploader.value.values()))  # Get the first uploaded file
    #    t_name_without_ext = Path(uploaded_file).stem

        file_content = uploaded_file['content']  # Extract file content as bytes
    
        # Convert bytes to a file-like object
        file_buffer = io.BytesIO(file_content)
    
        # Read CSV from the buffer
        tdf = pd.read_csv(file_buffer, sep="\t")
    #    t_name_without_ext = Path(tdf).stem

    #    print(tdf.head())  # Show the first few rows
        file_name = uploaded_file['metadata']['name']  # Extract filename
    
        print(f"Uploaded File Name: {file_name}")
    #    file_name_without_ext = Path(tdf).stem

        file_name_no_ext = Path(file_name).stem  # Remove extension

        print(f"File Name (Without Extension): {file_name_no_ext}")

        print(type(tdf))
        
    # Convert to JSON
    json_training_data = tdf.to_json(orient="records")
    #print(json_data)


    # Save the JSON file
    #with open("squad_sample_train.json", "w") as json_file:
    with open(f"{file_name_no_ext}.json", "w") as json_training_file:
        json_training_file.write(json_training_data)

    print(f"Conversion complete: {file_name_no_ext}.json created")

In [77]:
"""# Convert to JSON
json_training_data = tdf.to_json(orient="records")
#print(json_data)


# Save the JSON file
#with open("squad_sample_train.json", "w") as json_file:
with open(f"{t_name_without_ext}.json", "w") as json_training_file:
    json_training_file.write(json_training_data)

print(f"Conversion complete: {file_name_no_ext}.json created")"""

'# Convert to JSON\njson_training_data = tdf.to_json(orient="records")\n#print(json_data)\n\n\n# Save the JSON file\n#with open("squad_sample_train.json", "w") as json_file:\nwith open(f"{t_name_without_ext}.json", "w") as json_training_file:\n    json_training_file.write(json_training_data)\n\nprint(f"Conversion complete: {file_name_no_ext}.json created")'

In [79]:
import pandas as pd
from pathlib import Path

# Load the TSV file
#df = pd.read_csv("squad_sample_train.tsv", sep="\t")


tf = input("Please enter a training data filename.")
tdf = pd.read_csv(tf, sep="\t")
print(type(tdf))
#print(df)
t_name_without_ext = Path(tf).stem

# Convert to JSON
json_training_data = tdf.to_json(orient="records", indent=4)
#print(json_data)

# Save the JSON file
#with open("squad_sample_train.json", "w") as json_file:
with open(f"{t_name_without_ext}.json", "w") as json_training_file:
    json_training_file.write(json_training_data)

print(f"Conversion complete: {t_name_without_ext}.json created")

Please enter a training data filename. squad_sample_train.tsv


<class 'pandas.core.frame.DataFrame'>
Conversion complete: squad_sample_train.json created


In [81]:
# Load the TSV file
#df = pd.read_csv("squad_sample_validation.tsv", sep="\t")


vf = input("Please enter a validation data filename.")
vdf = pd.read_csv(vf, sep="\t")
#print(df)
v_name_without_ext = Path(vf).stem

# Convert to JSON
json_validation_data = vdf.to_json(orient="records", indent=4)
#print(json_data)

# Save the JSON file
#with open("squad_sample_train.json", "w") as json_file:
with open(f"{v_name_without_ext}.json", "w") as json_validation_file:
    json_validation_file.write(json_validation_data)

print(f"Conversion complete: {v_name_without_ext}.json created")

Please enter a validation data filename. squad_sample_validation.tsv


Conversion complete: squad_sample_validation.json created


In [83]:
if isinstance(json_training_data, str):  
    json_training_data = json.loads(json_training_data)  


#def insert_context_placeholders(training_data, id, title=title, context, question, answer):
def insert_context_placeholders_tr(training_data, id=None, title=None,  contexts=None, question=None, answers=None):

    id = []
    title = []
    contxt = []
    question = []
    answ = []

    for entry in training_data:
#        print(entry)
#        print(entry["title"])

        v = entry["id"]
#        print(v)
        id.append(v)
        v = entry["title"]
#        print(v)
        title.append(v)
        v = entry["question"]
#        print(v)
        question.append(v)
        context = entry["context"]
        answers = entry["answers"]
        # Find the position of the answer
        answer_start = context.find(answers)
        answer_end = answer_start + len(answers)

#        print(answer_start)
#        print(answer_end)

        # Tokenize while preserving position info
        tokens = tokenizer(context, return_offsets_mapping=True, return_tensors="pt")

#        print(tokens)

        # Find token-level positions for the answer
        start_token, end_token = None, None
        for idx, (start, end) in enumerate(tokens["offset_mapping"].squeeze().tolist()):
            if start == answer_start:
                start_token = idx
            if end == answer_end:
                end_token = idx
                break  # Stop once we find the end token
        answers = {
                "text": [answers],
                "answer_start": [answer_start]
            }
#   abs print(answers)
        answ.append(answers)
        contxt.append(context)
    training_dataset = Dataset.from_dict({
        "id": id,
        "title": title,
        "context": contxt,
        "question": question,
        "answers": answ
    })

#    print("Training dataset: ",training_dataset)
    
    return training_dataset
    
insert_context_placeholders_tr(json_training_data, [], [], [], [], [])

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5
})

In [85]:
training_dataset = insert_context_placeholders_tr(json_training_data, [], [], [], [], [])
#print(training_dataset)

In [87]:
if isinstance(json_validation_data, str):  
    json_validation_data = json.loads(json_validation_data)  


def insert_context_placeholders_val(validation_data, id=None, title=None,  contexts=None, question=None, answers=None):

    id = []
    title = []
    contxt = []
    question = []
    answ = []

    for entry in validation_data:
#        print(entry)
#        print(entry["title"])

        v = entry["id"]
#        print(v)
        id.append(v)
        v = entry["title"]
#        print(v)
        title.append(v)
        v = entry["question"]
#        print(v)
        question.append(v)
        context = entry["context"]
        answers = entry["answers"]
        # Find the position of the answer
        answer_start = context.find(answers)
        answer_end = answer_start + len(answers)

#        print(answer_start)
#        print(answer_end)

        # Tokenize while preserving position info
        tokens = tokenizer(context, return_offsets_mapping=True, return_tensors="pt")

#        print(tokens)

        # Find token-level positions for the answer
        start_token, end_token = None, None
        for idx, (start, end) in enumerate(tokens["offset_mapping"].squeeze().tolist()):
            if start == answer_start:
                start_token = idx
            if end == answer_end:
                end_token = idx
                break  # Stop once we find the end token
        answers = {
                "text": [answers],
                "answer_start": [answer_start]
            }
#   abs print(answers)
        answ.append(answers)
        contxt.append(context)
    validation_dataset = Dataset.from_dict({
        "id": id,
        "title": title,
        "context": contxt,
        "question": question,
        "answers": answ
    })

#    print("Validation dataset: ",validation_dataset)

    return validation_dataset
    
insert_context_placeholders_val(json_validation_data, [], [], [], [], [])

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 2
})

In [89]:
validation_dataset = insert_context_placeholders_val(json_validation_data, [], [], [], [], [])
#print(validation_dataset)

In [91]:
squad_sample = DatasetDict({
    "train": training_dataset,  # Add more splits (e.g., validation, test) as needed
    "validation": validation_dataset  # Add more splits (e.g., validation, test) as needed
})

In [93]:
validation_dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 2
})

In [95]:
import torch

def preprocess_function(examples):
    print(examples["question"])
    inputs = tokenizer(
        examples["question"], examples["context"],
        max_length=512, truncation=True, padding="max_length"
    )
    
    # Extract start and end positions
    start_positions = []
    end_positions = []
    for i, answer in enumerate(examples["answers"]):
        if len(answer["answer_start"]) > 0:
            start = answer["answer_start"][0]
            end = start + len(answer["text"][0])
        else:
            start = 0
            end = 0
        start = 0
        end = 0
        start_positions.append(start)
        end_positions.append(end)
    
    # Convert to PyTorch tensors
    inputs["input_ids"] = torch.tensor(inputs["input_ids"])
    inputs["attention_mask"] = torch.tensor(inputs["attention_mask"])
    inputs["start_positions"] = torch.tensor(start_positions)
    inputs["end_positions"] = torch.tensor(end_positions)
    
    return inputs

In [97]:
tokenized_squad = squad_sample.map(preprocess_function, batched=True, remove_columns=squad_sample["train"].column_names)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

['What is in front of the Notre Dame Main Building?', 'What is the Grotto at Notre Dame?', 'What is the primary seminary of the Congregation of the Holy Cross?', 'Which company is Danjaq, LLC associated with?', 'Which film studio won the full copyright film rights to Spectre?']


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

['Whose name was added to the name for every town or city with historical signiciance from the World War II period?', "What was the name of Tito's personal doctor?"]


In [99]:
# Set the format to PyTorch tensors
tokenized_squad.set_format(type="torch")

In [101]:
print(tokenized_squad)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 5
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 2
    })
})


In [103]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_squad["train"], batch_size=16, shuffle=True)
val_dataloader = DataLoader(tokenized_squad["validation"], batch_size=16)

In [105]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

C:\Users\Usuario\anaconda3\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [107]:
#import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):  # Number of epochs
    model.train()
    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        start_positions = batch["start_positions"].to(device)
        end_positions = batch["end_positions"].to(device)

        outputs = model(
            input_ids,
            attention_mask=attention_mask,
            start_positions=start_positions,
            end_positions=end_positions,
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Epoch {epoch}, Loss: {loss.item()}")


Epoch 0, Loss: 6.341365814208984
Epoch 1, Loss: 6.085456371307373
Epoch 2, Loss: 5.885241508483887


In [109]:
# Save the model
print("Saving the model...")
model.save_pretrained("./distilbert-squad7")
tokenizer.save_pretrained("./distilbert-squad7")
print("Model saved!")
print(type(model))

Saving the model...
Model saved!
<class 'transformers.models.distilbert.modeling_distilbert.DistilBertForQuestionAnswering'>


In [143]:
import torch
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering

# Load fine-tuned model (Replace with your actual model path if needed)
#model_name = "distilbert-base-uncased-distilled-squad"  # Or your fine-tuned checkpoint
#model_name = model("./distilbert-squad6")
#tokenizer = DistilBertTokenizerFast.from_pretrained("./distilbert-squad6")
#model_name = model
#tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
#model = DistilBertForQuestionAnswering.from_pretrained(model_name)

tokenizer = DistilBertTokenizerFast.from_pretrained("./distilbert-squad7")
model = DistilBertForQuestionAnswering.from_pretrained("./distilbert-squad7")

# Define your Question and Context
question = "What is the capital of France?"
context = "France's capital city is Paris. It is known for the Eiffel Tower."
#question = "What is the capital of Russia?"
#context = "Russia's capital city is not St Petersburg but Moscow. It is known for the Kremlin."

# Tokenize input
inputs = tokenizer(question, context, return_tensors="pt", truncation=True)
#print(inputs)

# Run the model to get start & end logits
with torch.no_grad():
    outputs = model(**inputs)

# Extract predicted answer positions
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Convert logits to token positions
start_idx = torch.argmax(start_logits)
end_idx = torch.argmax(end_logits) + 1  # +1 because slicing is exclusive

# Convert token indexes back to text
predicted_answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx])
)

print("Predicted Answer:", predicted_answer)


Predicted Answer: [CLS] what is the capital of france?


In [133]:
from transformers import pipeline

inputs = tokenizer(question, context, return_tensors="pt", truncation=True)


# Remove token_type_ids if present
#if "token_type_ids" in inputs:
#    del inputs["token_type_ids"]

# Pass the modified inputs to the model
#result = qa_pipeline({"question": question, "context": context})

# Create a QA pipeline for inference
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

Device set to use cpu


In [135]:
import evaluate
import numpy as np

# Load evaluation metrics from Hugging Face
metric = evaluate.load("squad")

def compute_metrics(predictions, references):
    return metric.compute(predictions=predictions, references=references)

In [137]:
from datasets import load_dataset
#dataset = load_dataset("squad", "rc", split="validation[:1]")  # Take 100 samples for quick testing
dataset = load_dataset("squad", split="validation[:1]")

In [138]:
# Format dataset for evaluation
predictions, references = [], []

for example in dataset:
    question = example["question"]
    context = example["context"]
    ground_truths = example["answers"]["text"]  # Extract answers
    answer_starts = example["answers"]["answer_start"]  # Extract answer positions

    # Get model prediction
    result = qa_pipeline({"question": question, "context": context})
#    result = qa_pipeline(question=question, context=context)  # Call the pipeline properly
    pred_answer = result["answer"]  # Extract only the predicted text

    # Store references in SQuAD format
    references.append({
        "id": example["id"],
        "answers": [{"text": ans, "answer_start": start} for ans, start in zip(ground_truths, answer_starts)]
    })

    # Store predictions
    predictions.append({"id": example["id"], "prediction_text": pred_answer})

# Compute EM and F1 scores
scores = compute_metrics(predictions, references)
print(scores)

C:\Users\Usuario\anaconda3\Lib\site-packages\transformers\pipelines\question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{'exact_match': 0.0, 'f1': 0.0}
